In [1]:
from importlib import reload
from helper import run
import ecc
import helper
import tx
import script

In [2]:
from ecc import PrivateKey, S256Point
from helper import decode_base58, SIGHASH_ALL, h160_to_p2pkh_address, hash160, h160_to_p2sh_address
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx, TxFetcher
from urllib.request import Request, urlopen
import json

In [3]:
privkey_1 = PrivateKey(int.from_bytes(b"Oscar Eduardo Serna Rosero","big"))
my_address_1 = privkey_1.point.address(testnet=True)
print(my_address_1)

mo3WWB4PoSHrudEBik1nUqfn1uZEPNYEc8


In [4]:
privkey_2 = PrivateKey(int.from_bytes(b"Oscar Eduardo Serna Rosero","little"))
my_address_2 = privkey_2.point.address(testnet=True)
print(my_address_2)

mwuCRN8bMLWN9ETs29U8KseDkau72M9byF


# Let's try to create a multisignature address that needs both signatures from privkey_1 and privkey_2

In [5]:
m=2
op_m = 82
n=2
op_n=82
length_pubkey=33
pubkey1=privkey_1.point.sec()
pubkey2=privkey_2.point.sec()

In [6]:
pubkey1.hex()

'03e07f96e5ba598431c0c994493a4ae988c9854c171d5d4bb140db0a27a4c853e4'

In [7]:
pubkey2.hex()

'031b63f964d8c65d1d1136dcfe5033dedea88c2d411934ea48c9708410be84e5ee'

In [8]:
#redeem_script = Script([op_m, length_pubkey, pubkey1, length_pubkey, pubkey2, op_n, 174])#174:OP_CHECKMULTISIG
redeem_script = Script([op_m, pubkey1, pubkey2, op_n, 174])#174:OP_CHECKMULTISIG
redeem_script

OP_2 03e07f96e5ba598431c0c994493a4ae988c9854c171d5d4bb140db0a27a4c853e4 031b63f964d8c65d1d1136dcfe5033dedea88c2d411934ea48c9708410be84e5ee OP_2 OP_CHECKMULTISIG

In [9]:
serialized_redeem = redeem_script.serialize()
serialized_redeem.hex()

'47522103e07f96e5ba598431c0c994493a4ae988c9854c171d5d4bb140db0a27a4c853e421031b63f964d8c65d1d1136dcfe5033dedea88c2d411934ea48c9708410be84e5ee52ae'

In [10]:
multi_sig_addr = h160_to_p2sh_address(hash160(serialized_redeem), testnet=True)
multi_sig_addr

'2MtjpwVyA7MYhBLirhcEhydEJNa7brhNojb'

# Now let's try to send some BTCs to this address

In [11]:
from script import p2sh_script
# Let's build the tx:
tx_outs =[]
tx_ins =[]

change = int(0.01 * 100000000)# we are going to fix this later
amountTx= int(0.01 * 100000000)

#It will send 0.01 to the multisig address
output = TxOut(amountTx, p2sh_script(decode_base58(multi_sig_addr)))
print(output)
tx_outs.append(output)

change_output = TxOut(change, p2pkh_script(decode_base58(my_address_1)))
print(change_output)
tx_outs.append(change_output)

prev1_tx_id = "ee337430b19e720f5b1bc4582df972bec0750fcfcbf6143d097121d0e11af478"
prev_tx1 = TxFetcher.fetch(prev1_tx_id, testnet=True)
print(prev_tx1)

def get_index(outs_list):
    for index,out in enumerate(outs_list):
        if out.script_pubkey.cmds[2] == decode_base58(my_address_1):
            return index
    return "output index not found"
    
prev1_index= get_index(prev_tx1.tx_outs)
#prev1_index=0
print(f"index 1: {prev1_index}")

tx_in1 = TxIn(bytes.fromhex(prev1_tx_id),prev1_index)
print(tx_in1)
tx_ins.append(tx_in1)

my_tx = Tx(1, tx_ins, tx_outs, 0, testnet=True)
print(my_tx)

# sign the inputs in the transaction object using the private key
for tx_input in range(len(tx_ins)):
    print(my_tx.sign_input(tx_input, privkey_1))
# print the transaction's serialization in hex

#Let's calculate the fee and the change:
tx_size = len(my_tx.serialize().hex())
fee_per_byte = 10 # I changed from 2 to 10 after sending this transaction because it had really low appeal to miners.
fee = tx_size * fee_per_byte
print(f"fee: {fee}")

change = prev_tx1.tx_outs[prev1_index].amount - fee - amountTx
print(f"change {change}")
#Let's make sure that we are actually spending the exact amount of the UTXO
total_send=fee+amountTx+change
diff = prev_tx1.tx_outs[prev1_index].amount-total_send
print(f"total {total_send}, diff: {diff}")


#Let's modify the amount in the change_output:
tx_outs[-1].amount = change

#we sign again because the hash changed
for tx_input in range(len(tx_ins)):
    print(my_tx.sign_input(tx_input, privkey_1))

print(my_tx.serialize().hex())

#for tx_input in range(len(tx_ins)):
 #   print(my_tx.verify_input(tx_input))




1000000:OP_HASH160 1060f2fd6058cd100fe4e485455badf453bd63e1 OP_EQUAL
1000000:OP_DUP OP_HASH160 52903efc1004de01883ba3687be2a8ea4f6b1b19 OP_EQUALVERIFY OP_CHECKSIG
tx: ee337430b19e720f5b1bc4582df972bec0750fcfcbf6143d097121d0e11af478
version: 2
tx_ins:
a84ffedf78eb2f940287a56a2fdc11aba60fc916853dbb5fd6e856e32dc6f576:0
tx_outs:
4179188255:OP_HASH160 d7c83330ba9762519b1e536cf2af01855d62ab74 OP_EQUAL
1758430:OP_DUP OP_HASH160 52903efc1004de01883ba3687be2a8ea4f6b1b19 OP_EQUALVERIFY OP_CHECKSIG
locktime: 1665844
index 1: 1
ee337430b19e720f5b1bc4582df972bec0750fcfcbf6143d097121d0e11af478:1
tx: e917cd2443af0b0b14e45c6daec853e465ac19e8750632ccd14e5aacc2c880ea
version: 1
tx_ins:
ee337430b19e720f5b1bc4582df972bec0750fcfcbf6143d097121d0e11af478:1
tx_outs:
1000000:OP_HASH160 1060f2fd6058cd100fe4e485455badf453bd63e1 OP_EQUAL
1000000:OP_DUP OP_HASH160 52903efc1004de01883ba3687be2a8ea4f6b1b19 OP_EQUALVERIFY OP_CHECKSIG
locktime: 0
OP_DUP OP_HASH160 52903efc1004de01883ba3687be2a8ea4f6b1b19 OP_EQUALVERIF

In [12]:
my_tx.verify_input(0)

OP_DUP OP_HASH160 52903efc1004de01883ba3687be2a8ea4f6b1b19 OP_EQUALVERIFY OP_CHECKSIG


True

In [13]:
li = ["a","b","c"]
print(*li)

a b c


# Now let's try to spend the UTXO from the multi-sig address

In [14]:
reload(tx)

<module 'tx' from '/Users/oscareduardosernarosero/Documents/US/Rice U/programmingbitcoin/code-ch08/tx.py'>

In [17]:
tx_outs =[]
tx_ins =[]

amountTx= int(0.01 * 100000000)# we'll fix this later after calculating the fee

#It will send back all the money to our address_1 minus the fee
output = TxOut(amountTx,  p2pkh_script(decode_base58(my_address_1)))
print(output)
tx_outs.append(output)


prev1_tx_id = "9f6e65900ca76e8351b7babc6bbc2e5b0b2ded99e4e6f5b1ec74c2a190abf31a"
prev_tx1 = TxFetcher.fetch(prev1_tx_id, testnet=True)
print(prev_tx1)

def get_index(outs_list, address):
    print(address)
    print(decode_base58(address).hex())
    print(outs_list)
    for index,out in enumerate(outs_list):
        if out.script_pubkey.cmds[2] == decode_base58(address) or out.script_pubkey.cmds[1] == decode_base58(address):
            return index
    return "output index not found"
    
prev1_index= get_index(prev_tx1.tx_outs, multi_sig_addr)
#prev1_index=0
print(f"index 1: {prev1_index}")

tx_in1 = TxIn(bytes.fromhex(prev1_tx_id),prev1_index)
print(tx_in1)
tx_ins.append(tx_in1)

my_multisig_tx = Tx(1, tx_ins, tx_outs, 0, testnet=True)
print(my_multisig_tx)



# sign the inputs in the transaction object using the private key
for tx_input in range(len(tx_ins)):
    print(my_multisig_tx.sign_input_multisig(tx_input, [privkey_1, privkey_2], redeem_script))#redeem_script from earlier
# print the transaction's serialization in hex

#Let's calculate the fee and the change:
tx_size = len(my_multisig_tx.serialize().hex())
fee_per_byte = 25 # I changed from 10 to 25 after sending this transaction because apparently it is more expensive for multisig tx.
fee = tx_size * fee_per_byte 
print(f"fee: {fee}")

amountTx = prev_tx1.tx_outs[prev1_index].amount - fee 
print(f"amountTx : {amountTx}")
#Let's make sure that we are actually spending the exact amount of the UTXO
total_send=fee+amountTx
diff = prev_tx1.tx_outs[prev1_index].amount-total_send
print(f"total {total_send}, diff: {diff}")


#Let's modify the amount in the change_output:
tx_outs[-1].amount = amountTx

#we sign again because the hash changed
for tx_input in range(len(tx_ins)):
    print(my_multisig_tx.sign_input_multisig(tx_input, [privkey_1, privkey_2], redeem_script))#redeem_script from earlier


print(my_multisig_tx.serialize().hex())

#for tx_input in range(len(tx_ins)):
 #   print(my_tx.verify_input(tx_input))

1000000:OP_DUP OP_HASH160 52903efc1004de01883ba3687be2a8ea4f6b1b19 OP_EQUALVERIFY OP_CHECKSIG
tx: 9f6e65900ca76e8351b7babc6bbc2e5b0b2ded99e4e6f5b1ec74c2a190abf31a
version: 1
tx_ins:
ee337430b19e720f5b1bc4582df972bec0750fcfcbf6143d097121d0e11af478:1
tx_outs:
1000000:OP_HASH160 1060f2fd6058cd100fe4e485455badf453bd63e1 OP_EQUAL
753950:OP_DUP OP_HASH160 52903efc1004de01883ba3687be2a8ea4f6b1b19 OP_EQUALVERIFY OP_CHECKSIG
locktime: 0
2MtjpwVyA7MYhBLirhcEhydEJNa7brhNojb
1060f2fd6058cd100fe4e485455badf453bd63e1
[1000000:OP_HASH160 1060f2fd6058cd100fe4e485455badf453bd63e1 OP_EQUAL, 753950:OP_DUP OP_HASH160 52903efc1004de01883ba3687be2a8ea4f6b1b19 OP_EQUALVERIFY OP_CHECKSIG]
index 1: 0
9f6e65900ca76e8351b7babc6bbc2e5b0b2ded99e4e6f5b1ec74c2a190abf31a:0
tx: a716747d4c63bcc054ab55d2baa20ae4d50ff74a0ec31f6d7a6efc48b5438547
version: 1
tx_ins:
9f6e65900ca76e8351b7babc6bbc2e5b0b2ded99e4e6f5b1ec74c2a190abf31a:0
tx_outs:
1000000:OP_DUP OP_HASH160 52903efc1004de01883ba3687be2a8ea4f6b1b19 OP_EQUALVERIFY OP

In [16]:
from io import BytesIO
rebuilt_script = Script.parse(BytesIO(bytes.fromhex("522103e07f96e5ba598431c0c994493a4ae988c9854c171d5d4bb140db0a27a4c853e421031b63f964d8c65d1d1136dcfe5033dedea88c2d411934ea48c9708410be84e5ee52ae")))

IndexError: index out of range

In [ ]:
rebuilt_script

In [ ]:
rebuilt_script = Script.parse(BytesIO(bytes.fromhex("5221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152ae")))
rebuilt_script